In [1]:
import pandas as pd
import numpy as np
import re
import json
import copy

In [2]:
# read in CSV
df = pd.read_csv('data_2018.csv')

# filter only MS, MD, WS, WD, XD
df = df[df['Discipline'].isin(['MS', 'MD', 'WS', 'WD', 'XD'])].reset_index()

In [3]:
def update_tournament_type(x):
    if ('1000' in x) or ('Finals' in x) or ('Grade 1' in x):
        return 'WTFSuper1000'
    elif ('750' in x):
        return 'Super750'
    elif('500' in x):
        return 'Super500'
    elif('300' in x):
        return 'Super300'
    elif ('100' in x):
        return 'Super100'
    elif('International' in x):
        return 'ISIC'
    else:
        return 'Continental'
        
df['Tournament Type New'] = df['Tournament Type'].apply(lambda x: update_tournament_type(x))

In [4]:
df.head()

,index,Player 1 Link,Player 1 Name,Player 2 Link,Player 2 Name,Tournament Link,Tournament Name,Tournament Type,Tournament City,Tournament Country,...,Player 2 Sequence,Match Score,Match Statistic,Game Statistic,Retired,Winner,Player 1 Seed,Player 2 Seed,Match Date,Tournament Type New
0,0,['http://bwf.tournamentsoftware.com/player-pro...,"['Niclas NOHR', 'Sara THYGESEN']",['http://bwf.tournamentsoftware.com/player-pro...,"['Peter KAESBAUER', 'Olga KONON']",http://bwf.tournamentsoftware.com/sport/tourna...,YONEX Swiss Open 2018,HSBC BWF World Tour Super 300,Basel,Switzerland,...,"[[0, 0, 1, 2, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5,...","[[21, 16], [19, 21], [21, 11]]","[[8, 4], [3, 2], [109, 109], [61, 48]]","[[[4, 3], [2, 0], [37, 37], [21, 16]], [[3, 4]...",False,1,NaN,NaN,2/20/2018,Super300
1,1,['http://bwf.tournamentsoftware.com/player-pro...,"['Robin TABELING', 'Cheryl SEINEN']",['http://bwf.tournamentsoftware.com/player-pro...,"['Gaetan MITTELHEISSER', 'Emilie LEFEL']",http://bwf.tournamentsoftware.com/sport/tourna...,YONEX Swiss Open 2018,HSBC BWF World Tour Super 300,Basel,Switzerland,...,"[[0, 1, 1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, 9, 9,...","[[19, 21], [21, 16], [21, 9]]","[[7, 4], [2, 1], [107, 107], [61, 46]]","[[[6, 4], [0, 1], [40, 40], [19, 21]], [[3, 4]...",False,1,NaN,NaN,2/20/2018,Super300
2,2,['http://bwf.tournamentsoftware.com/player-pro...,"['Tinn ISRIYANET', 'Pacharapun CHOCHUWONG']",['http://bwf.tournamentsoftware.com/player-pro...,"['Jakub BITMAN', 'Alzbeta BASOVA']",http://bwf.tournamentsoftware.com/sport/tourna...,YONEX Swiss Open 2018,HSBC BWF World Tour Super 300,Basel,Switzerland,...,"[[0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 4, 5, 5, 5,...","[[21, 11], [21, 11]]","[[7, 3], [4, 0], [64, 64], [42, 22]]","[[[5, 3], [2, 0], [32, 32], [21, 11]], [[7, 3]...",False,1,NaN,NaN,2/20/2018,Super300
3,3,['http://bwf.tournamentsoftware.com/player-pro...,"['Thom GICQUEL', 'Delphine DELRUE']",['http://bwf.tournamentsoftware.com/player-pro...,"['Ben LANE', 'Jessica PUGH']",http://bwf.tournamentsoftware.com/sport/tourna...,YONEX Swiss Open 2018,HSBC BWF World Tour Super 300,Basel,Switzerland,...,"[[0, 1, 2, 2, 2, 2, 3, 4, 5, 6, 6, 6, 6, 6, 7,...","[[19, 21], [15, 21]]","[[4, 5], [0, 2], [76, 76], [34, 42]]","[[[4, 4], [0, 1], [40, 40], [19, 21]], [[4, 5]...",False,2,NaN,6,2/20/2018,Super300
4,4,['http://bwf.tournamentsoftware.com/player-pro...,"['Jacco ARENDS', 'Selena PIEK']",['http://bwf.tournamentsoftware.com/player-pro...,"['YANG Ming-Tse', 'SUNG Shuo Yun']",http://bwf.tournamentsoftware.com/sport/tourna...,YONEX Swiss Open 2018,HSBC BWF World Tour Super 300,Basel,Switzerland,...,"[[0, 0, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,...","[[21, 16], [21, 12]]","[[7, 5], [3, 0], [70, 70], [42, 28]]","[[[7, 5], [1, 0], [37, 37], [21, 16]], [[6, 2]...",False,1,5,NaN,2/20/2018,Super300


# Convert to JSON format for Force-Directed Graph

In [5]:
nodes = {}
links = {}

In [6]:
def create_all_nodes(df, nodes):
    for idx in range(0, df.shape[0]):
        
        # singles
        if df.iloc[idx]['Discipline'] == 'MS' or df.iloc[idx]['Discipline'] == 'WS':
            p1 = df.iloc[idx]['Player 1 Name'].replace(',', '').strip()
            p2 = df.iloc[idx]['Player 2 Name'].replace(',', '').strip() 
            winner = df.iloc[idx]['Winner']
            
            if (p1 in nodes):
                if (winner == 1):
                    nodes[p1]['Wins'] += 1
                    nodes[p1]['Played'] += 1
                else:
                    nodes[p1]['Played'] += 1
            else:
                nodes[p1] = {'Discipline': df.iloc[idx]['Discipline'], 
                                   'Country': df.iloc[idx][str('Player 1 Country')],
                                   'Wins': 0, 'Played': 1}
                if (winner == 1):
                    nodes[p1]['Wins'] = 1
            
            if (p2 in nodes):
                if (winner == 2):
                    nodes[p2]['Wins'] += 1
                    nodes[p2]['Played'] += 1
                else:
                    nodes[p2]['Played'] += 1
            else:
                nodes[p2] = {'Discipline': df.iloc[idx]['Discipline'], 
                                   'Country': df.iloc[idx][str('Player 2 Country')],
                                   'Wins': 0, 'Played': 1}
                if (winner == 2):
                    nodes[p2]['Wins'] = 1
                        
        # Doubles
        else:
            p1 = df.iloc[idx]['Player 1 Name'].replace("'", '').strip()
            p2 = df.iloc[idx]['Player 2 Name'].replace("'", '').strip()
            
            # broken records in p1 (comma other than teammate)
            if ('LAU Yu Ming' in p1) or ('LAU Yu Leong' in p1) or ('LIM Peiy Yee' in p1) or ('Desiree' in p1):
                p1 = p1.replace('LAU Yu Ming,', 'LAU Yu Ming')
                p1 = p1.replace('LAU Yu Leong,', 'LAU Yu Leong')
                p1 = p1.replace('LIM Peiy Yee,', 'LAU Yu Leong')
                p1 = p1.replace('Desiree,', 'Desiree')

            # broken records in p2 (comma other than teammate)
            if ('LAU Yu Ming' in p2) or ('LAU Yu Leong' in p2) or ('LIM Peiy Yee' in p2) or ('Desiree' in p2):
                p2 = p2.replace('LAU Yu Ming,', 'LAU Yu Ming')
                p2 = p2.replace('LAU Yu Leong,', 'LAU Yu Leong')
                p2 = p2.replace('LIM Peiy Yee,', 'LAU Yu Leong')
                p2 = p2.replace('Desiree,', 'Desiree')
            
            split1 = p1.split(',')
            split2 = p2.split(',')
            
            p11 = split1[0].replace("[","").strip()
            p12 = split1[1].replace("]","").strip()
            p21 = split2[0].replace("[","").strip()
            p22 = split2[1].replace("]","").strip()
            
            p1 = p11 + "," + p12
            p1_rev = p12 + "," + p11
            p2 = p21 + "," + p22
            p2_rev = p22 + "," + p21
            
            winner = df.iloc[idx]['Winner']
                      
            # for player 1
            if (p1 in nodes):
                if (winner == 1):
                    nodes[p1]['Wins'] += 1
                    nodes[p1]['Played'] += 1
                else:
                    nodes[p1]['Played'] += 1
            elif (p1_rev in nodes):
                if (winner == 1):
                    nodes[p1_rev]['Wins'] += 1
                    nodes[p1_rev]['Played'] += 1
                else:
                    nodes[p1_rev]['Played'] += 1
            else:
                country = df.iloc[idx][str('Player 1 Country')].replace("'", '').strip().split(',')
                c1 = country[0].replace("[","").strip()
                c2 = country[1].replace("]","").strip()
                
                nodes[p1] = {'Discipline': df.iloc[idx]['Discipline'], 
                               'Country': c1 + "," + c2,
                               'Wins': 0, 'Played': 1}
                if (winner == 1):
                    nodes[p1]['Wins'] = 1
            
            # for player 2
            if (p2 in nodes):
                if (winner == 1):
                    nodes[p2]['Wins'] += 1
                    nodes[p2]['Played'] += 1
                else:
                    nodes[p2]['Played'] += 1
                    
            elif (p2_rev in nodes):
                if (winner == 1):
                    nodes[p2_rev]['Wins'] += 1
                    nodes[p2_rev]['Played'] += 1
                else:
                    nodes[p2_rev]['Played'] += 1
                    
            else:
                country = df.iloc[idx][str('Player 2 Country')].replace("'", '').strip().split(',')
                c1 = country[0].replace("[","").strip()
                c2 = country[1].replace("]","").strip()
                
                nodes[p2] = {'Discipline': df.iloc[idx]['Discipline'], 
                               'Country': c1 + "," + c2,
                               'Wins': 0, 'Played': 1}
                if (winner == 2):
                    nodes[p2]['Wins'] = 1
    
    return nodes

In [7]:
nodes = create_all_nodes(df, nodes)

In [8]:
nodes

{'Niclas NOHR,Sara THYGESEN': {'Discipline': 'XD',
  'Country': 'Denmark,Denmark',
  'Wins': 15,
  'Played': 35},
 'Peter KAESBAUER,Olga KONON': {'Discipline': 'XD',
  'Country': 'Germany,Germany',
  'Wins': 17,
  'Played': 32},
 'Robin TABELING,Cheryl SEINEN': {'Discipline': 'XD',
  'Country': 'Netherlands,Netherlands',
  'Wins': 12,
  'Played': 30},
 'Gaetan MITTELHEISSER,Emilie LEFEL': {'Discipline': 'XD',
  'Country': 'France,France',
  'Wins': 0,
  'Played': 2},
 'Tinn ISRIYANET,Pacharapun CHOCHUWONG': {'Discipline': 'XD',
  'Country': 'Thailand,Thailand',
  'Wins': 8,
  'Played': 10},
 'Jakub BITMAN,Alzbeta BASOVA': {'Discipline': 'XD',
  'Country': 'Czech Republic,Czech Republic',
  'Wins': 10,
  'Played': 24},
 'Thom GICQUEL,Delphine DELRUE': {'Discipline': 'XD',
  'Country': 'France,France',
  'Wins': 16,
  'Played': 34},
 'Ben LANE,Jessica PUGH': {'Discipline': 'XD',
  'Country': 'England,England',
  'Wins': 11,
  'Played': 22},
 'Jacco ARENDS,Selena PIEK': {'Discipline': 'XD

In [9]:
links = {}

In [10]:
def create_all_links(df, links):
    for idx in range(0, df.shape[0]):
        winner = df.iloc[idx]['Winner']
        
        # singles
        if df.iloc[idx]['Discipline'] == 'MS' or df.iloc[idx]['Discipline'] == 'WS':
            p1 = df.iloc[idx]['Player 1 Name'].replace(',', '').strip()
            p2 = df.iloc[idx]['Player 2 Name'].replace(',', '').strip() 
            
        # doubles
        else:
            p1 = df.iloc[idx]['Player 1 Name'].replace("'", '').strip()
            p2 = df.iloc[idx]['Player 2 Name'].replace("'", '').strip()
            
            # broken records in p1 (comma other than teammate)
            if ('LAU Yu Ming' in p1) or ('LAU Yu Leong' in p1) or ('LIM Peiy Yee' in p1) or ('Desiree' in p1):
                p1 = p1.replace('LAU Yu Ming,', 'LAU Yu Ming')
                p1 = p1.replace('LAU Yu Leong,', 'LAU Yu Leong')
                p1 = p1.replace('LIM Peiy Yee,', 'LAU Yu Leong')
                p1 = p1.replace('Desiree,', 'Desiree')

            # broken records in p2 (comma other than teammate)
            if ('LAU Yu Ming' in p2) or ('LAU Yu Leong' in p2) or ('LIM Peiy Yee' in p2) or ('Desiree' in p2):
                p2 = p2.replace('LAU Yu Ming,', 'LAU Yu Ming')
                p2 = p2.replace('LAU Yu Leong,', 'LAU Yu Leong')
                p2 = p2.replace('LIM Peiy Yee,', 'LAU Yu Leong')
                p2 = p2.replace('Desiree,', 'Desiree')
            
            split1 = p1.split(',')
            split2 = p2.split(',')
            
            p11 = split1[0].replace("[","").strip()
            p12 = split1[1].replace("]","").strip()
            p21 = split2[0].replace("[","").strip()
            p22 = split2[1].replace("]","").strip()
            
            # check if nodes exist
            if (p11 + "," + p12 in nodes):
                p1 = p11 + "," + p12
            elif(p12 + "," + p11 in nodes):
                p1 = p12 + "," + p11
            else:
                print('error')
            
            if (p21 + "," + p22 in nodes):
                p2 = p21 + "," + p22
            elif(p22 + "," + p21 in nodes):
                p2 = p22 + "," + p21
            else:
                print('error')

        # determine order of insertion. Winner is first
        if (winner == 1):
            link = str(p1) + '#' + str(p2)
            rev= str(p2) + '#' + str(p1)
        else:
            link = str(p2) + '#' + str(p1)
            rev = str(p1) + '#' + str(p2)

        if (link in links):
            # if exist in dictionary increment wins by 1
            links[link] += 1
        elif (rev in links):
            # if revese exist in dictionary decrement by 1
            links[rev] -= 1
        else:
            # create new
            links[link] = 1
        
    return links

In [11]:
links = create_all_links(df, links)

In [12]:
links

{'Niclas NOHR,Sara THYGESEN#Peter KAESBAUER,Olga KONON': 2,
 'Robin TABELING,Cheryl SEINEN#Gaetan MITTELHEISSER,Emilie LEFEL': 1,
 'Tinn ISRIYANET,Pacharapun CHOCHUWONG#Jakub BITMAN,Alzbeta BASOVA': 1,
 'Ben LANE,Jessica PUGH#Thom GICQUEL,Delphine DELRUE': 0,
 'Jacco ARENDS,Selena PIEK#YANG Ming-Tse,SUNG Shuo Yun': 1,
 'Marvin SEIDEL,Linda EFLER#HOO Pang Ron,PECK Yen Wei': 1,
 'Bastian KERSAUDY,Lea PALERMO#Oliver SCHALLER,Celine BURKART': 2,
 'CHANG Ko-Chi,CHENG Chi Ya#Andreas HEINZ,Annika HORBACH': 1,
 'Ruben JILLE,Imke VAN DER AAR#Yehezkiel Fritz MAINAKY,Lyanny Alessandra MAINAKY': 1,
 'Jones Ralfy JANSEN,Carla NYENHUIS#Freek GOLINSKI,Lise JAQUES': 1,
 'Mark LAMSFUSS,Isabel HERTTRICH#William VILLEGER,Sharone BAUER': 1,
 'Marcus ELLIS,Lauren SMITH#Mathias BAY-SMIDT,Rikke SØBY': 2,
 'Philipp DISCHER#Mathew FOGARTY': 1,
 'Lars SCHAENZLER#Yehezkiel Fritz MAINAKY': 1,
 'Joel KÖNIG#Michael SPENCER-SMITH': 1,
 'Riichi TAKESHITA#Michal ROGALSKI': 1,
 'Søren TOFT#Alexander ROOVERS': 1,
 'GURU

In [13]:
def top_k(k, nodes, links, discipline):
    node_data = []
    links_data = []
    
    # players ranking data from https://bwfbadminton.com/rankings/2/bwf-world-rankings/6/men-s-singles/2018/52/?rows=25&page_no=1
    if (discipline == 'MS'):
        players = ['Kento MOMOTA', 'SHI Yu Qi', 'CHOU Tien Chen', 'CHEN Long', 'SON Wan Ho', 'Viktor AXELSEN', 
                   'Anthony Sinisuka GINTING', 'KIDAMBI Srikanth', 'Tommy SUGIARTO', 'Kenta NISHIMOTO', 'Jonatan CHRISTIE',
                   'Sameer VERMA', 'LIN Dan', 'NG Ka Long Angus', 'Kantaphon WANGCHAROEN']
    elif (discipline == 'WS'):
        players = ['TAI Tzu Ying', 'Nozomi OKUHARA', 'PUSARLA V. Sindhu', 'CHEN Yu Fei', 'Akane YAMAGUCHI', 
                   'Carolina MARIN', 'HE Bing Jiao', 'Ratchanok INTANON', 'Saina NEHWAL', 'Beiwen ZHANG', 
                   'SUNG Ji Hyun', 'Sayaka TAKAHASHI', 'Michelle LI', 'Nitchaon JINDAPOL', 'Gregoria Mariska TUNJUNG']
    elif (discipline == 'MD'):
        players = ['Marcus Fernaldi GIDEON,Kevin Sanjaya SUKAMULJO', 'LI Jun Hui,LIU Yu Chen', 'Takeshi KAMURA,Keigo SONODA',
                   'CHEN Hung Ling,WANG Chi-Lin', 'Hiroyuki ENDO,Yuta WATANABE', 'Kim ASTRUP,Anders Skaarup RASMUSSEN', 
                   'Fajar ALFIAN,Muhammad Rian ARDIANTO', 'HAN Cheng Kai,ZHOU Hao Dong', 'Mohammad AHSAN,Hendra SETIAWAN',
                   'LIU Cheng,ZHANG Nan', 'Takuto INOUE,Yuki KANEKO','LIAO Min Chun,SU Ching Heng', 'HE Ji Ting,TAN Qiang',
                   'GOH V Shem,TAN Wee Kiong', 'Mads CONRAD-PETERSEN,Mads Pieler KOLDING']
    elif (discipline == 'WD'):
        players = ['Yuki FUKUSHIMA,Sayaka HIROTA','Misaki MATSUTOMO,Ayaka TAKAHASHI','Mayu MATSUMOTO,Wakana NAGAHARA',
                   'Greysia POLII,Apriyani RAHAYU','CHEN Qing Chen,JIA Yi Fan', 'LEE So Hee,SHIN Seung Chan','Shiho TANAKA,Koharu YONEMOTO',
                  'Jongkolphan KITITHARAKUL,Rawinda PRAJONGJAI','Gabriela STOEVA,Stefani STOEVA','Ayako SAKURAMOTO,Yukiko TAKAHATA',
                  'DU Yue,LI Yin Hui','Della Destiara HARIS,Rizki Amelia PRADIPTA','Nami MATSUYAMA,Chiharu SHIDA','Maiken FRUERGAARD,Sara THYGESEN',
                  'Naoko FUKUMAN,Kurumi YONAO']
    elif(discipline == 'XD'):
        players = ['ZHENG Si Wei,HUANG Ya Qiong','WANG Yi Lyu,HUANG Dong Ping','Yuta WATANABE,Arisa HIGASHINO','Tontowi AHMAD,Liliyana NATSIR',
                  'Dechapol PUAVARANUKROH,Sapsiree TAERATTANACHAI','CHAN Peng Soon,GOH Liu Ying','TANG Chun Man,TSE Ying Suet','Mathias CHRISTIANSEN,Christinna PEDERSEN',
                  'Chris ADCOCK,Gabrielle ADCOCK', 'ZHANG Nan,LI Yin Hui','GOH Soon Huat,LAI Shevon Jemie', 'Hafiz FAIZAL,Gloria Emanuelle WIDJAJA','HE Ji Ting,DU Yue',
                  'Marcus ELLIS,Lauren SMITH','Praveen JORDAN,Melati Daeva OKTAVIANTI']
    
    players = players[0:k]
    
    # based on player rankings obtained from website
    temp = pd.DataFrame.from_dict(nodes, orient = 'index').reset_index()
    temp = temp[temp['Discipline'] == discipline]
    temp = temp[temp['index'].isin(players)]
    
      
      # based on top k number on wins on data
#     temp = pd.DataFrame.from_dict(nodes, orient = 'index')
#     temp = temp.sort_values(by = ['Wins'], ascending = False)
#     temp = temp.groupby(['Discipline']).head(100).reset_index()
#     temp = temp[temp['Discipline'] == discipline]
#     players = temp['index'].values

    
    for i in range(temp.shape[0]):
        node_data.append({"id": temp.iloc[i]['index'], "Country": temp.iloc[i]['Country'], "Wins": int(temp.iloc[i]['Wins']),
                         "Played": int(temp.iloc[i]['Played'])})
    
    for ind, elem in enumerate(players):
        for ind2, elem2 in enumerate(players[ind+1:]):
            ord1 = elem + "#" + elem2
            ord2 = elem2 + "#" + elem
            if ord1 in links:
                if links[ord1] < 0:
                    links_data.append({"source": elem, "target": elem2, "value": int(links[ord1] * -1)})
                else:
                    links_data.append({"source": elem2, "target": elem, "value": int(links[ord1]) })
            elif ord2 in links:
                if links[ord2] < 0:
                    links_data.append({"source": elem2, "target": elem, "value": int(links[ord2] * -1)})
                else:
                    links_data.append({"source": elem, "target": elem2, "value": int(links[ord2]) })
                    
    return node_data, links_data

In [14]:
node_data_MS, links_data_MS = top_k(15, nodes, links, 'MS')
node_data_WS, links_data_WS = top_k(15, nodes, links, 'WS')
node_data_MD, links_data_MD = top_k(15, nodes, links, 'MD')
node_data_WD, links_data_WD = top_k(15, nodes, links, 'WD')
node_data_XD, links_data_XD = top_k(15, nodes, links, 'XD')

In [15]:
final = [{"nodes": node_data_MS, "links": links_data_MS},
         {"nodes": node_data_WS, "links": links_data_WS},
         {"nodes": node_data_MD, "links": links_data_MD},
         {"nodes": node_data_WD, "links": links_data_WD},
         {"nodes": node_data_XD, "links": links_data_XD}]

In [16]:
final

[{'nodes': [{'id': 'Anthony Sinisuka GINTING',
    'Country': 'Indonesia',
    'Wins': 24,
    'Played': 40},
   {'id': 'CHEN Long', 'Country': 'China', 'Wins': 25, 'Played': 36},
   {'id': 'CHOU Tien Chen',
    'Country': 'Chinese Taipei',
    'Wins': 46,
    'Played': 63},
   {'id': 'Jonatan CHRISTIE',
    'Country': 'Indonesia',
    'Wins': 21,
    'Played': 37},
   {'id': 'KIDAMBI Srikanth', 'Country': 'India', 'Wins': 22, 'Played': 34},
   {'id': 'Kantaphon WANGCHAROEN',
    'Country': 'Thailand',
    'Wins': 27,
    'Played': 48},
   {'id': 'Kenta NISHIMOTO', 'Country': 'Japan', 'Wins': 24, 'Played': 39},
   {'id': 'Kento MOMOTA', 'Country': 'Japan', 'Wins': 55, 'Played': 62},
   {'id': 'LIN Dan', 'Country': 'China', 'Wins': 19, 'Played': 36},
   {'id': 'NG Ka Long Angus',
    'Country': 'Hong Kong China',
    'Wins': 19,
    'Played': 34},
   {'id': 'SHI Yu Qi', 'Country': 'China', 'Wins': 39, 'Played': 50},
   {'id': 'SON Wan Ho', 'Country': 'Korea', 'Wins': 31, 'Played': 46},


In [17]:
# write to JSON File
with open('graph.json', 'w', encoding='utf-8') as f:
    json.dump(final, f, ensure_ascii=False, indent=4)

# Convert to Map Format (Player Count)

In [18]:
country_df = pd.DataFrame.from_dict(nodes, orient = 'index').reset_index()

In [19]:
dict_converter = {'Malaysia': 'MYS', 'Scotland': 'GBR', 'India': 'IND', 'Jordan': 'JOR', 'United Arab Emirates': 'UAE',
                  'Korea': 'KOR', 'Thailand': 'THA' , 'Ethiopia': 'ETH', 'Canada': 'CAN', 'England': 'GBR', 'Egypt': 'EGY',
                  'Indonesia': 'IDN', 'Uzbekistan': 'UZB', 'New Zealand': 'NZL', 'U.S.A.': 'USA', 'Spain': 'ESP',
                  'Czech Republic': 'CZE', 'Poland': 'POL', 'Sweden': 'SWE', 'Hungary': 'HUN', 'Ireland': 'IRL', 
                  'Wales': 'GBR', 'Azerbaijan': 'AZE', 'Algeria': 'DZA', 'France': 'FRA', 'Switzerland': 'CHE',
                  'Bahrain': 'BHR', 'Portugal': 'PRT', 'Belgium': 'BEL', 'Brunei Darussalem': 'BRN', 'Turkey': 'TUR',
                  'Norway': 'NOR', 'Philippines': 'PHL', 'Finland': 'FIN', 'Uganda': 'UGA', 'Japan': 'JPN', 'Estonia': 'EST',
                  'Russia': 'RUS', 'Latvia': 'LVA', 'Belarus': 'BLR', 'Italy': 'ITA', 'Bulgaria': 'BGR', 'Israel': 'ISR',
                  'Germany': 'DEU', 'Moldova': 'MDA', 'Denmark': 'DNK', 'Iran': 'IRN', 'Syrian Arab Republic': 'SYR',
                  'Netherlands': 'NLD', 'Iraq': 'IRQ', 'Ukraine': 'UKR', 'Maldives': 'MDV', 'Cyprus': 'CYP', 'Slovenia': 'SVN',
                  'Serbia': 'SRB', 'Pakistan': 'PAK', 'Australia': 'AUS', 'Austria': 'AUT', 'Kazakhstan': 'KAZ',
                  'Mexico': 'MEX', 'China': 'CHN', 'Vietnam': 'VNM', 'Mongolia': 'MNG', 'Nepal': 'NPL', 'Morocco': 'MAR',
                  'Singapore': 'SGP', 'Croatia': 'HRV', 'Peru': 'PER', 'Hong Kong China': 'CHN',
                  'Chinese Taipei': 'TWN', 'Gibraltar': 'GBR', 'Romania': 'ROU', 'Barbados': 'BRB', 'Iceland': 'ISL',
                  'Sri Lanka': 'LKA', 'South Africa': 'ZAF', 'Nigeria': 'NGA', 'Brazil': 'BRA', 'Malta': 'MLT',
                  'Armenia': 'ARM', 'Greece': 'GRC', 'Mauritius': 'MUS', 'Lithuania': 'LTU', 'Macau China': 'CHN',
                  'North Macedonia': 'MKD', 'Luxembourg': 'LUX', 'Slovakia': 'SVK', 'Jamaica': 'JAM',
                  'Guatemala': 'GCA', 'Ecuador': 'ECU', 'Suriname': 'SUR', 'Lao': 'LAO', 'Trinidad and Tobago': 'TTO',
                  'Myanmar': 'MMR', 'Chile': 'CHL', 'French Polynesia (Tahiti)': 'PYF', 'Benin': 'BEN'}

In [20]:
# initialize everything to 0
countries_count = {}
for elem in dict_converter.values():
    countries_count[elem] = 0

players_countries_count = {'MS': countries_count.copy(), 'WS': countries_count.copy(), 'MD': countries_count.copy(), 
                           'WD': countries_count.copy(), 'XD': countries_count.copy()}

In [21]:
# generate count
def country_count(x):
    splits = x['Country'].split(',')
    discipline = x['Discipline']
    
    if len(splits) == 1:
        country1 = dict_converter[splits[0]]
        players_countries_count[discipline][country1] += 1
    else:
        country1 = dict_converter[splits[0]]
        country2 = dict_converter[splits[1]]
        players_countries_count[discipline][country1] += 0.5
        players_countries_count[discipline][country2] += 0.5

country_df.apply(lambda x: country_count(x), axis = 1);

In [22]:
players_countries_count

{'MS': {'MYS': 37,
  'GBR': 65,
  'IND': 70,
  'JOR': 3,
  'UAE': 9,
  'KOR': 22,
  'THA': 17,
  'ETH': 0,
  'CAN': 26,
  'EGY': 4,
  'IDN': 54,
  'UZB': 3,
  'NZL': 5,
  'USA': 32,
  'ESP': 26,
  'CZE': 17,
  'POL': 26,
  'SWE': 29,
  'HUN': 11,
  'IRL': 11,
  'AZE': 4,
  'DZA': 3,
  'FRA': 20,
  'CHE': 11,
  'BHR': 3,
  'PRT': 14,
  'BEL': 14,
  'BRN': 1,
  'TUR': 13,
  'NOR': 4,
  'PHL': 1,
  'FIN': 20,
  'UGA': 0,
  'JPN': 28,
  'EST': 17,
  'RUS': 26,
  'LVA': 5,
  'BLR': 0,
  'ITA': 7,
  'BGR': 8,
  'ISR': 3,
  'DEU': 24,
  'MDA': 3,
  'DNK': 48,
  'IRN': 5,
  'SYR': 1,
  'NLD': 12,
  'IRQ': 0,
  'UKR': 23,
  'MDV': 1,
  'CYP': 2,
  'SVN': 10,
  'SRB': 5,
  'PAK': 5,
  'AUS': 9,
  'AUT': 8,
  'KAZ': 2,
  'MEX': 4,
  'CHN': 32,
  'VNM': 9,
  'MNG': 1,
  'NPL': 4,
  'MAR': 1,
  'SGP': 8,
  'HRV': 4,
  'PER': 4,
  'TWN': 32,
  'ROU': 1,
  'BRB': 1,
  'ISL': 2,
  'LKA': 2,
  'ZAF': 0,
  'NGA': 0,
  'BRA': 2,
  'MLT': 3,
  'ARM': 0,
  'GRC': 3,
  'MUS': 1,
  'LTU': 0,
  'MKD': 0,
  'L

In [23]:
final = pd.DataFrame.from_dict(players_countries_count).reset_index()
final = pd.melt(final, id_vars=['index'], value_vars=['MS', 'WS', 'MD', 'WD', 'XD'], var_name='discipline', value_name='player_count')

In [24]:
final.head()

,index,discipline,player_count
0,ARM,MS,0.0
1,AUS,MS,9.0
2,AUT,MS,8.0
3,AZE,MS,4.0
4,BEL,MS,14.0


In [25]:
# write to CSV
final.to_csv('player_count.csv', index=False)

# Tournament Winner + Country

In [26]:
dict_converter = {'Malaysia': 'MYS', 'Scotland': 'GBR', 'India': 'IND', 'Jordan': 'JOR', 'United Arab Emirates': 'UAE',
                  'Korea': 'KOR', 'Thailand': 'THA' , 'Ethiopia': 'ETH', 'Canada': 'CAN', 'England': 'GBR', 'Egypt': 'EGY',
                  'Indonesia': 'IDN', 'Uzbekistan': 'UZB', 'New Zealand': 'NZL', 'U.S.A.': 'USA', 'Spain': 'ESP',
                  'Czech Republic': 'CZE', 'Poland': 'POL', 'Sweden': 'SWE', 'Hungary': 'HUN', 'Ireland': 'IRL', 
                  'Wales': 'GBR', 'Azerbaijan': 'AZE', 'Algeria': 'DZA', 'France': 'FRA', 'Switzerland': 'CHE',
                  'Bahrain': 'BHR', 'Portugal': 'PRT', 'Belgium': 'BEL', 'Brunei Darussalem': 'BRN', 'Turkey': 'TUR',
                  'Norway': 'NOR', 'Philippines': 'PHL', 'Finland': 'FIN', 'Uganda': 'UGA', 'Japan': 'JPN', 'Estonia': 'EST',
                  'Russia': 'RUS', 'Latvia': 'LVA', 'Belarus': 'BLR', 'Italy': 'ITA', 'Bulgaria': 'BGR', 'Israel': 'ISR',
                  'Germany': 'DEU', 'Moldova': 'MDA', 'Denmark': 'DNK', 'Iran': 'IRN', 'Syrian Arab Republic': 'SYR',
                  'Netherlands': 'NLD', 'Iraq': 'IRQ', 'Ukraine': 'UKR', 'Maldives': 'MDV', 'Cyprus': 'CYP', 'Slovenia': 'SVN',
                  'Serbia': 'SRB', 'Pakistan': 'PAK', 'Australia': 'AUS', 'Austria': 'AUT', 'Kazakhstan': 'KAZ',
                  'Mexico': 'MEX', 'China': 'CHN', 'Vietnam': 'VNM', 'Mongolia': 'MNG', 'Nepal': 'NPL', 'Morocco': 'MAR',
                  'Singapore': 'SGP', 'Croatia': 'HRV', 'Peru': 'PER', 'Hong Kong China': 'CHN',
                  'Chinese Taipei': 'TWN', 'Gibraltar': 'GBR', 'Romania': 'ROU', 'Barbados': 'BRB', 'Iceland': 'ISL',
                  'Sri Lanka': 'LKA', 'South Africa': 'ZAF', 'Nigeria': 'NGA', 'Brazil': 'BRA', 'Malta': 'MLT',
                  'Armenia': 'ARM', 'Greece': 'GRC', 'Mauritius': 'MUS', 'Lithuania': 'LTU', 'Macau China': 'CHN',
                  'North Macedonia': 'MKD', 'Luxembourg': 'LUX', 'Slovakia': 'SVK', 'Jamaica': 'JAM',
                  'Guatemala': 'GCA', 'Ecuador': 'ECU', 'Suriname': 'SUR', 'Lao': 'LAO', 'Trinidad and Tobago': 'TTO',
                  'Myanmar': 'MMR', 'Chile': 'CHL', 'French Polynesia (Tahiti)': 'PYF', 'Benin': 'BEN'}

In [27]:
winner_country_count = {}


# initialize everything to 0
countries_count = {}
for elem in dict_converter.values():
    countries_count[elem] = {'Continental': 0, 'ISIC': 0, 'Super100': 0, 'Super300': 0, 'Super500': 0, 'Super750': 0, 'WTFSuper1000': 0}

winner_country_count = {'MS': copy.deepcopy(countries_count), 'WS': copy.deepcopy(countries_count), 'MD': copy.deepcopy(countries_count), 
                           'WD': copy.deepcopy(countries_count), 'XD': copy.deepcopy(countries_count)}

In [28]:
def check_last_day(x):
    if (x['Tournament End Date'] == x['Match Date']):
        return "yes"
    else:
        return "no"
    
df['last_day'] = df.apply(lambda x: check_last_day(x), axis = 1)

In [29]:
def create_tournament_winners(discipline):
    temp = df[(df['last_day'] == "yes") & (df['Discipline'] == discipline)].reset_index()
    temp_dict = {}
    
    for i in range(temp.shape[0]):
        winner = temp.iloc[i,:]['Winner']
        tname = temp.iloc[i,:]['Tournament Name']
        pwinner = temp.iloc[i,:]['Player ' + str(winner) + ' Name'].replace("[", '').replace("]", '').replace("'", '').replace(", ",",").strip()
        cwinner = temp.iloc[i,:]['Player ' + str(winner) + ' Country'].replace("[", '').replace("]", '').replace("'", '').replace(", ",",").strip()
    
        if tname in temp_dict:
            # hardcode two rows
            if (tname == 'Tarragona 2018 Mediterranean Games') and discipline == 'MS':
                temp_dict[tname] = {'Winner Name': 'Pablo ABIAN', 'Winner Country': 'Spain', 'temp': {}}
            elif (tname == 'Tarragona 2018 Mediterranean Games') and discipline == 'WS':
                temp_dict[tname] = {'Winner Name': 'Neslihan YIGIT', 'Winner Country': 'Turkey'}
            else:
                if pwinner in temp_dict[tname]['temp']:
                    temp_dict[tname]['temp'][pwinner] += 1
                else:
                    temp_dict[tname]['temp'][pwinner] = 1

                # fix semifinals matchwinners
                if (len(temp_dict[tname]['temp']) == 2) & (np.sum(list(temp_dict[tname]['temp'].values())) == 3):
                    mydict = temp_dict[tname]['temp']
                    temp_dict[tname]['Winner Name'] = list(mydict.keys())[list(mydict.values()).index(2)]
                    temp_dict[tname]['Winner Country'] = nodes[temp_dict[tname]['Winner Name']]['Country']

        else:
            temp_dict[tname] = {'Winner Name': pwinner, 
                                'Winner Country': cwinner,
                                'temp': {pwinner: 1} }
    
    temp_summary = pd.DataFrame.from_dict(temp_dict, orient = 'index').reset_index() 
    uniq_tournaments = df[['Tournament Name', 'Tournament Type', 'Tournament Type New']].drop_duplicates().reset_index()
    temp_summary = pd.merge(temp_summary, uniq_tournaments, left_on = 'index', right_on = 'Tournament Name')
        
    for ind in range(temp_summary.shape[0]):
        splits = temp_summary.loc[ind]['Winner Country'].split(',')
        tour_type = temp_summary.loc[ind]['Tournament Type New']
        if len(splits) == 1:
            winner_country_count[discipline][dict_converter[splits[0]]][tour_type] += 1
        else:
            winner_country_count[discipline][dict_converter[splits[0]]][tour_type] += 0.5
            winner_country_count[discipline][dict_converter[splits[1]]][tour_type] += 0.5

In [30]:
create_tournament_winners('MS')
create_tournament_winners('WS')
create_tournament_winners('MD')
create_tournament_winners('WD')
create_tournament_winners('XD')

In [31]:
winner_country_count

{'MS': {'MYS': {'Continental': 0,
   'ISIC': 2,
   'Super100': 0,
   'Super300': 1,
   'Super500': 0,
   'Super750': 1,
   'WTFSuper1000': 0},
  'GBR': {'Continental': 0,
   'ISIC': 2,
   'Super100': 0,
   'Super300': 0,
   'Super500': 0,
   'Super750': 0,
   'WTFSuper1000': 0},
  'IND': {'Continental': 0,
   'ISIC': 2,
   'Super100': 4,
   'Super300': 2,
   'Super500': 0,
   'Super750': 0,
   'WTFSuper1000': 0},
  'JOR': {'Continental': 0,
   'ISIC': 0,
   'Super100': 0,
   'Super300': 0,
   'Super500': 0,
   'Super750': 0,
   'WTFSuper1000': 0},
  'UAE': {'Continental': 0,
   'ISIC': 0,
   'Super100': 0,
   'Super300': 0,
   'Super500': 0,
   'Super750': 0,
   'WTFSuper1000': 0},
  'KOR': {'Continental': 0,
   'ISIC': 0,
   'Super100': 0,
   'Super300': 3,
   'Super500': 1,
   'Super750': 0,
   'WTFSuper1000': 0},
  'THA': {'Continental': 0,
   'ISIC': 0,
   'Super100': 1,
   'Super300': 0,
   'Super500': 0,
   'Super750': 0,
   'WTFSuper1000': 0},
  'ETH': {'Continental': 0,
   'ISI

In [32]:
final = pd.DataFrame.from_dict(winner_country_count).reset_index()
final = pd.melt(final, id_vars=['index'], value_vars=['MS', 'WS', 'MD', 'WD', 'XD'], var_name='discipline', value_name='win_count')
temp = final['win_count'].apply(pd.Series)
final = final.join(temp)
final.drop(['win_count'], axis = 1, inplace = True)

In [33]:
final.head()

,index,discipline,Continental,ISIC,Super100,Super300,Super500,Super750,WTFSuper1000
0,ARM,MS,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AUS,MS,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AUT,MS,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AZE,MS,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BEL,MS,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# write to CSV
final.to_csv('country_win.csv', index=False)

# Tree Chart Format

In [35]:
def top_k_tree(k, nodes, discipline):    
    tree_data = []
    
    # Player Ranking Data: https://bwfbadminton.com/rankings/2/bwf-world-rankings/6/men-s-singles/2018/52/?rows=25&page_no=1
    if (discipline == 'MS'):
        players = ['Kento MOMOTA', 'SHI Yu Qi', 'CHOU Tien Chen', 'CHEN Long', 'SON Wan Ho', 'Viktor AXELSEN', 
                   'Anthony Sinisuka GINTING', 'KIDAMBI Srikanth', 'Tommy SUGIARTO', 'Kenta NISHIMOTO', 'Jonatan CHRISTIE',
                   'Sameer VERMA', 'LIN Dan', 'NG Ka Long Angus', 'Kantaphon WANGCHAROEN']
        prize = ['$683,439.00', '$428,267.50', '$456,018.50', '$1,169,402.50', '$507,082.50','$538,127.00','$211,130.00',
                '$440,702.50', '$329,051.00', '$88,458.75', '$115,930.50', '$124,513.50','$966,242.50','$168,421.00','$46,411.25']
        record = ['263-72', '171-59','313-177','398-94','330-176','297-116','140-82','210-102','331-192','138-76','165-81','165-81','164-70','638-108','216-124','119-77']
    elif (discipline == 'WS'):
        players = ['TAI Tzu Ying', 'Nozomi OKUHARA', 'PUSARLA V. Sindhu', 'CHEN Yu Fei', 'Akane YAMAGUCHI', 
                   'Carolina MARIN', 'HE Bing Jiao', 'Ratchanok INTANON', 'Saina NEHWAL', 'Beiwen ZHANG', 
                   'SUNG Ji Hyun', 'Sayaka TAKAHASHI', 'Michelle LI', 'Nitchaon JINDAPOL', 'Gregoria Mariska TUNJUNG']
        prize = ['$1,119,755.00', '$597,915.50', '$586,958.00', '$285,320.00', '$669,995.00', '$591,428.50', '$278,007.50',
                 '$712,192.50', '$798,247.50', '$208,988.62', '$640,897.50', '$165,479.50', '$131,517.00', '$155,993.00', '$50,973.75']
        record = ['339-136','257-82','293-120','159-65','253-89','331-101','169-93','337-155','411-176','228-116','348-178','199-82','289-133','211-143','130-53']
    elif (discipline == 'MD'):
        players = ['Marcus Fernaldi GIDEON,Kevin Sanjaya SUKAMULJO', 'LI Jun Hui,LIU Yu Chen', 'Takeshi KAMURA,Keigo SONODA',
                   'CHEN Hung Ling,WANG Chi-Lin', 'Hiroyuki ENDO,Yuta WATANABE', 'Kim ASTRUP,Anders Skaarup RASMUSSEN', 
                   'Fajar ALFIAN,Muhammad Rian ARDIANTO', 'HAN Cheng Kai,ZHOU Hao Dong', 'Mohammad AHSAN,Hendra SETIAWAN',
                   'LIU Cheng,ZHANG Nan', 'Takuto INOUE,Yuki KANEKO','LIAO Min Chun,SU Ching Heng', 'HE Ji Ting,TAN Qiang',
                   'GOH V Shem,TAN Wee Kiong', 'Mads CONRAD-PETERSEN,Mads Pieler KOLDING']
        prize = ['$1,089,206.09', '$538,498.76', '$444,040.23', '$281,085.73', '$339,180.23', '$271,094.25',
                 '$135,121.62', '$148,962.52','$860,546.26', '$425,891.25', '$147,168.62', '$122,949.60', '$78,355.00','$386,870.57', '$284,339.89']
        record = ['172-42','176-79','207-111','132-90','45-30','176-105','115-53','55-32','190-59','74-32','116-72','73-36','45-29','154-72','195-94']
    elif (discipline == 'WD'):
        players = ['Yuki FUKUSHIMA,Sayaka HIROTA','Misaki MATSUTOMO,Ayaka TAKAHASHI','Mayu MATSUMOTO,Wakana NAGAHARA',
                   'Greysia POLII,Apriyani RAHAYU','CHEN Qing Chen,JIA Yi Fan', 'LEE So Hee,SHIN Seung Chan','Shiho TANAKA,Koharu YONEMOTO',
                  'Jongkolphan KITITHARAKUL,Rawinda PRAJONGJAI','Gabriela STOEVA,Stefani STOEVA','Ayako SAKURAMOTO,Yukiko TAKAHATA',
                  'DU Yue,LI Yin Hui','Della Destiara HARIS,Rizki Amelia PRADIPTA','Nami MATSUYAMA,Chiharu SHIDA','Maiken FRUERGAARD,Sara THYGESEN',
                  'Naoko FUKUMAN,Kurumi YONAO']
        prize = ['$535,372.73', '$1,190,837.24','$267,784.98','$436,975.64','$553,880.01','$648,356.85','$301,473.82',
                 '$171,212.88','$190,667.28','$112,153.63','$141,795.02','$155,756.75','$76,242.99','$56,439.77','$188,739.98']
        record = ['150-50','358-125','94-39','64-26','149-47','136-70','105-49','126-89','243-103','69-16','36-26','42-27','48-20','83-75','135-88']
    elif(discipline == 'XD'):
        players = ['ZHENG Si Wei,HUANG Ya Qiong','WANG Yi Lyu,HUANG Dong Ping','Yuta WATANABE,Arisa HIGASHINO','Tontowi AHMAD,Liliyana NATSIR',
                  'Dechapol PUAVARANUKROH,Sapsiree TAERATTANACHAI','CHAN Peng Soon,GOH Liu Ying','TANG Chun Man,TSE Ying Suet','Mathias CHRISTIANSEN,Christinna PEDERSEN',
                  'Chris ADCOCK,Gabrielle ADCOCK', 'ZHANG Nan,LI Yin Hui','GOH Soon Huat,LAI Shevon Jemie', 'Hafiz FAIZAL,Gloria Emanuelle WIDJAJA','HE Ji Ting,DU Yue',
                  'Marcus ELLIS,Lauren SMITH','Praveen JORDAN,Melati Daeva OKTAVIANTI']
        prize = ['$1,085,180.60','$444,841.89','$224,366.88','$1,247,899.99','$188,064.37','$419,371.62','$248,304.26','$568,965.25','$455,656.01','$731,837.50','$124,538.81',
                '$131,212.37','$116,513.14','$93,783.13','$248,392.10']
        record = ['76-6','85-27','65-41','307-86','101-47','243-137','71-35','36-17','243-137','60-21','68-35','33-23','56-26','69-24','25-17']
    
    players = players[0:k]
    prize = prize[0:k]
    record = record[0:k]
    win_perc = []
    for i in record:
        splits = i.split('-')
        perc = (float(splits[0]) / (float(splits[0]) + float(splits[1]))) * 100
        perc = str(round(perc,1)) + "%"
        win_perc.append(perc)
    

    temp = pd.DataFrame.from_dict(nodes, orient = 'index').reset_index()
    
    for ind, elem in enumerate(players):
        a = {"data": {"Name": "#" + str(ind+1) + " " + elem, "Prize": prize[ind], 
                      "Nationality": temp[temp['index'] == elem]['Country'].values[0], 
                      "Win_Perc": win_perc[ind],
                      "Career_Record": record[ind],
                      "Parent": discipline},
            "height": 1, "depth": 2, "id": elem}
        tree_data.append(a)
    
    return tree_data

In [36]:
tree_data_MS = top_k_tree(15, nodes, 'MS')
tree_data_WS = top_k_tree(15, nodes, 'WS')
tree_data_MD = top_k_tree(15, nodes, 'MD')
tree_data_WD = top_k_tree(15, nodes, 'WD')
tree_data_XD = top_k_tree(15, nodes, 'XD')

In [37]:
master = {"data": {"Name": "Badminton"}, "height": 3, "depth": 0, "parent": "null", "id": "Badminton",
          "children": [
              {"data": {"Name": "Men's Singles"}, "height": 2, "depth": 1, "parent": "Badminton", "id": "MS", "children": tree_data_MS},
              {"data": {"Name": "Women's Singles"}, "height": 2, "depth": 1, "parent": "Badminton", "id": "WS", "children": tree_data_WS},
              {"data": {"Name": "Mens's Doubles"}, "height": 2, "depth": 1, "parent": "Badminton", "id": "MD", "children": tree_data_MD},
              {"data": {"Name": "Women's Doubles"}, "height": 2, "depth": 1, "parent": "Badminton", "id": "WD", "children": tree_data_WD},
              {"data": {"Name": "Mixed Doubles"}, "height": 2, "depth": 1, "parent": "Badminton", "id": "XD", "children": tree_data_XD},
          ]}

In [38]:
# write to JSON File
with open('tree.json', 'w', encoding='utf-8') as f:
    json.dump(master, f, ensure_ascii=False, indent=4)

In [39]:
master

{'data': {'Name': 'Badminton'},
 'height': 3,
 'depth': 0,
 'parent': 'null',
 'id': 'Badminton',
 'children': [{'data': {'Name': "Men's Singles"},
   'height': 2,
   'depth': 1,
   'parent': 'Badminton',
   'id': 'MS',
   'children': [{'data': {'Name': '#1 Kento MOMOTA',
      'Prize': '$683,439.00',
      'Nationality': 'Japan',
      'Win_Perc': '78.5%',
      'Career_Record': '263-72',
      'Parent': 'MS'},
     'height': 1,
     'depth': 2,
     'id': 'Kento MOMOTA'},
    {'data': {'Name': '#2 SHI Yu Qi',
      'Prize': '$428,267.50',
      'Nationality': 'China',
      'Win_Perc': '74.3%',
      'Career_Record': '171-59',
      'Parent': 'MS'},
     'height': 1,
     'depth': 2,
     'id': 'SHI Yu Qi'},
    {'data': {'Name': '#3 CHOU Tien Chen',
      'Prize': '$456,018.50',
      'Nationality': 'Chinese Taipei',
      'Win_Perc': '63.9%',
      'Career_Record': '313-177',
      'Parent': 'MS'},
     'height': 1,
     'depth': 2,
     'id': 'CHOU Tien Chen'},
    {'data': {'Name